In [29]:
#Algunos datos de utilidad según conclusiones del TP1

#En qué horarios se realizan más subastas?
# Teniendo en cuenta solo los días hábiles
# 5~10    : < 200k
# 4,11~20 : 200k~300k
# 20~0   : 300k ~ 400k
# 1~2    : > 400k

# :. entre las 20~2 hs se realizan la mayor cantidad de subastas con picos entre 1~2 hs.

In [30]:
# Depende un evento o una conversión de un usuario del día de la semana? Y de la hora?

#La cantidad de eventos se mantine constantes, excepto en los días M y X, donde encontramos un pico.
#Teniendo en cuenta la hora, la mayor cantidad de eventos se produce durante la tarde noche, inicio de la madrugada:
#Pico(+175k):    [0,1]
#Alta(150k~100k):(1,4)U[15,0)
#Media(100k~50k): [4,15)
#Baja(0~25k):(7,9)
#:. La mayor cantidad de eventos sucede durante el rango 15~1(tarde noche)

#INSTALACIONES
# La mayor cantidad se realiza los días M y X
# Los días M y X se duplica la cantidad de instalaciones aproximadamente(350~400)vs(750~800)
# Respecto del horario: Comportamiento similar a los eventos; la mayor cantidad se registran durante la tarde~noche(13~3)
# Afecta la presencia de wifi a la cantidad de instalaciones?
#El 80% de las instalaciones se lleva a cabo con dispositivos usando wifi.
#Cualés son las app mas populares?
# app_id | users | events
# 66       70k      325k
# 63       18k      181k 
# 145      17.5k    252k
# 64       17k      259k
# 103       15k     137k

## Predicciones
- En un instante dado, estimar St(d) el tiempo hasta que un dispositivo d aparezca de vuelta en una subasta RTB
- En un instante dado, estimar Sc(d) el tiempo hasta que un dispositivo d convierta



# Arboles - xgboost

In [31]:
import pandas as pd
import matplotlib as mp
import numpy as np
from datetime import timedelta
import datetime as dt
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

%matplotlib inline

## Preparación de datos

## Configuracion

In [123]:
#Root de archivos de datos
root_dir = '../data/'
#Lee menos data si es testing
is_testing = True
fraccion_a_usar = 0.01

print('Root path data: '+root_dir)
print('Es testing: '+str(is_testing))
print('Faccion de registros a usar para testing: '+str(fraccion_a_usar))
# Cantidad minima de aparicion de equipos en un dataframe. Esta variable se usa para eliminar los registros
# que tengan pocas apariciones dentro del set de datos total.
min_devices = 10
print('Min de device a tener en cuenta: '+str(min_devices))

# Identificador unico de equipos
device_uuid = ['ref_type','ref_hash']

# Ventanas de tiempo
windows = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': dt.datetime(2019,4,18),
'max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23),dt.datetime(2019,4,24)]
})

Root path data: ../data/
Es testing: True
Faccion de registros a usar para testing: 0.01
Min de device a tener en cuenta: 10


In [124]:
windows

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-18,1,2
2,2019-04-23,2019-04-18,1,3
3,2019-04-24,2019-04-18,1,4


In [34]:
#Se lee el dataframe de auctions

#Optimizado para menos memoria
auction_dtypes = {
    'ref_type_id': np.int8,
    'device_id': np.int64,
    'source_id': np.int8
}
#root_dir + 'auctions.csv.gzip'
auctions = pd.read_csv(root_dir+'auctions.csv.gzip',
                       compression = 'gzip',
                       dtype = auction_dtypes,
                       parse_dates = ['date'])
auctions.rename({'device_id':'ref_hash',
                 'ref_type_id':'ref_type'}, inplace=True, axis='columns')

# Para sumar los registros
auctions['n'] = 1

In [35]:
# Info de registros totales/a usar para entrenar
print('Total records:',auctions['n'].sum())
if is_testing:
    auctions = auctions.sample(frac=fraccion_a_usar)
    print("Is Testing, #records:", auctions['n'].sum())
else:
    print("Not Testing, #records:", auctions['n'].sum())

('Total records:', 47409528)
('Is Testing, #records:', 474095)


In [36]:
to_predict = pd.read_csv(root_dir+'target_competencia_ids.csv')
#predict reaparicion=st
predict_ids = to_predict.apply(lambda x: np.int64(x['ref_hash'][0:x['ref_hash'].find('_')]), axis='columns').drop_duplicates().to_frame()
#predict conversion=sc
predict_ids.head()

,0
0,1000169251625791246
2,1000395625957344683
4,1003027494996471685
6,1006670001679961544
8,1007573308966476713


### Arreglo de datos en auctions

In [37]:
auctions.head(3)

,date,ref_hash,ref_type,source_id,n
5008935,2019-04-21 15:38:48.883211,735982827853855852,1,1,1
663483,2019-04-19 05:48:26.263505,7263427207369556942,1,1,1
28660809,2019-04-25 04:51:03.882531,2207318136648149188,1,7,1


In [38]:
# Se eliminan los reg con pocas apariciones dentro del set de datos
print('Minimo de devices a usar '+str(min_devices))
orig_count = auctions['n'].sum()
auctions_clean = auctions.groupby(device_uuid, sort = False).filter(lambda x: x['n'].sum() >= min_devices) 
last_count = auctions_clean['n'].sum()
print('Cantidad auctions originales '+str(orig_count))
print('Cantidad auctions eliminados '+str(orig_count-last_count))
print('Cantidad auctions restantes '+str(last_count))

Minimo de devices a usar 10
Cantidad auctions originales 474095
Cantidad auctions eliminados 287955
Cantidad auctions restantes 186140


In [39]:
auctions_clean.head()

,date,ref_hash,ref_type,source_id,n
5008935,2019-04-21 15:38:48.883211,735982827853855852,1,1,1
39370812,2019-04-23 02:57:30.405789,334825238317164912,1,1,1
22577208,2019-04-19 06:19:40.588747,6663761678917706060,1,3,1
46691078,2019-04-18 22:15:49.707233,4550303430425129899,1,3,1
4224951,2019-04-26 16:47:26.170351,1105908109143296216,1,0,1


In [40]:
device_uuid+['date']

['ref_type', 'ref_hash', 'date']

In [41]:
auctions_clean.dtypes

date         datetime64[ns]
ref_hash              int64
ref_type               int8
source_id              int8
n                     int64
dtype: object

In [42]:
auc_clean = auctions_clean.sort_values(by=device_uuid+['date'], inplace=False)
#setea el próximo día de aparición(next_date)
auc_clean['next_date'] = auc_clean.groupby(device_uuid, as_index = False, sort=False)['date'].transform(lambda x: x.shift(-1))
# saco los registros 'borde', aquellos que no tienen 'next_date'
auc_clean = auc_clean.loc[(~auc_clean['next_date'].isnull())]
# Seteo los segundos de las apariciones
auc_clean['secs_to_next'] = (auc_clean['next_date'] - auc_clean['date']).transform(lambda x: round(x.total_seconds()))
auc_clean.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next
2012559,2019-04-19 18:26:55.718385,1666627460470179,1,1,1,2019-04-19 18:29:58.659713,183.0
9684262,2019-04-19 18:29:58.659713,1666627460470179,1,1,1,2019-04-21 05:10:20.220516,124822.0
24408718,2019-04-21 05:10:20.220516,1666627460470179,1,0,1,2019-04-21 05:16:52.546128,392.0
23735860,2019-04-21 05:16:52.546128,1666627460470179,1,0,1,2019-04-22 07:13:58.262799,93426.0
41719679,2019-04-22 07:13:58.262799,1666627460470179,1,1,1,2019-04-22 07:18:28.680238,270.0


In [43]:
auc_clean['hs_to_next'] = (auc_clean['secs_to_next'] ).transform(lambda x: x/(60*60))
auc_clean.head()

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next,hs_to_next
2012559,2019-04-19 18:26:55.718385,1666627460470179,1,1,1,2019-04-19 18:29:58.659713,183.0,0.050833
9684262,2019-04-19 18:29:58.659713,1666627460470179,1,1,1,2019-04-21 05:10:20.220516,124822.0,34.672778
24408718,2019-04-21 05:10:20.220516,1666627460470179,1,0,1,2019-04-21 05:16:52.546128,392.0,0.108889
23735860,2019-04-21 05:16:52.546128,1666627460470179,1,0,1,2019-04-22 07:13:58.262799,93426.0,25.951667
41719679,2019-04-22 07:13:58.262799,1666627460470179,1,1,1,2019-04-22 07:18:28.680238,270.0,0.075000


In [44]:
auc_clean['n'].sum()

176722

In [45]:
auc_clean['date'].min()

Timestamp('2019-04-18 00:00:00.674118')

In [46]:
auc_clean['date'].max()

Timestamp('2019-04-26 23:57:21.063375')

# Armado de features

In [47]:
windows.set_index('window_nr')

,max_date,min_date,n
window_nr,,,
1,2019-04-21,2019-04-18,1
2,2019-04-22,2019-04-18,1
3,2019-04-23,2019-04-18,1
4,2019-04-24,2019-04-18,1


## Rearmado de ventanas

In [48]:
windows2 = pd.DataFrame({
'window_nr':[1,2,3,4],
'n': 1,
'min_date': [dt.datetime(2019,4,18),dt.datetime(2019,4,19),dt.datetime(2019,4,20),dt.datetime(2019,4,21)],
'max_date': [dt.datetime(2019,4,21),dt.datetime(2019,4,22),dt.datetime(2019,4,23),dt.datetime(2019,4,24)]
})
windows2

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-19,1,2
2,2019-04-23,2019-04-20,1,3
3,2019-04-24,2019-04-21,1,4


In [49]:
cutoff_date = windows2.set_index('window_nr').loc[1,'max_date']
cutoff_date

Timestamp('2019-04-21 00:00:00')

In [50]:
train_auctions = auc_clean.loc[auctions['date'] < cutoff_date]
train_auctions.head(3)
train_auctions.count()

date            61008
ref_hash        61008
ref_type        61008
source_id       61008
n               61008
next_date       61008
secs_to_next    61008
hs_to_next      61008
dtype: int64

In [93]:
train_auctions.head(3)

,date,ref_hash,ref_type,source_id,n,next_date,secs_to_next,hs_to_next
2012559,2019-04-19 18:26:55.718385,1666627460470179,1,1,1,2019-04-19 18:29:58.659713,183.0,0.050833
9684262,2019-04-19 18:29:58.659713,1666627460470179,1,1,1,2019-04-21 05:10:20.220516,124822.0,34.672778
44470422,2019-04-18 10:44:10.904867,2417786714126835,1,3,1,2019-04-18 14:05:58.145714,12107.0,3.363056


In [51]:
device_uuid

['ref_type', 'ref_hash']

In [52]:
#Veo cuantos device tengo para el 'train_auctions'
devices = train_auctions[device_uuid].drop_duplicates()
devices['ref_hash'].count()

8227

In [53]:
#train_auctions[train_auctions['ref_hash'] == 2125435237741841]#13
#train_auctions.groupby(by='ref_hash')['secs_to_next'].sum().loc[2125435237741841,]/13#344042.0

In [55]:
#train_auctions2 = train_auctions[[]]
train_auc = train_auctions.reset_index()
train_auc = train_auc[['date','ref_hash','source_id','next_date','secs_to_next','hs_to_next']]
train_auc.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next
0,2019-04-19 18:26:55.718385,1666627460470179,1,2019-04-19 18:29:58.659713,183.0,0.050833
1,2019-04-19 18:29:58.659713,1666627460470179,1,2019-04-21 05:10:20.220516,124822.0,34.672778
2,2019-04-18 10:44:10.904867,2417786714126835,3,2019-04-18 14:05:58.145714,12107.0,3.363056
3,2019-04-18 14:05:58.145714,2417786714126835,1,2019-04-18 19:36:23.325793,19825.0,5.506944
4,2019-04-18 19:36:23.325793,2417786714126835,1,2019-04-19 06:28:35.369393,39132.0,10.870000


In [56]:
#Se desea saber el tiempo promedio entre arribos de los dispositivos a las encuestas.
temp = train_auctions.groupby(device_uuid, as_index='False')['secs_to_next'].mean().to_frame()['secs_to_next']
temp.columns = ['secs_to_next_mean']

In [57]:
temp=train_auctions.groupby(device_uuid, as_index='False')['secs_to_next'].mean().to_frame().reset_index()
temp=temp.rename(columns={'secs_to_next':'secs_to_next_mean'})
temp.head()

,ref_type,ref_hash,secs_to_next_mean
0,1,1666627460470179,62502.500000
1,1,2417786714126835,44029.000000
2,1,6380906313550611,48891.166667
3,1,7554663982392633,32794.600000
4,1,10954770305551958,42089.000000


In [58]:
train_auc2 = train_auc.merge(temp,how='inner',on='ref_hash')
train_auc2.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean
0,2019-04-19 18:26:55.718385,1666627460470179,1,2019-04-19 18:29:58.659713,183.0,0.050833,1,62502.5
1,2019-04-19 18:29:58.659713,1666627460470179,1,2019-04-21 05:10:20.220516,124822.0,34.672778,1,62502.5
2,2019-04-18 10:44:10.904867,2417786714126835,3,2019-04-18 14:05:58.145714,12107.0,3.363056,1,44029.0
3,2019-04-18 14:05:58.145714,2417786714126835,1,2019-04-18 19:36:23.325793,19825.0,5.506944,1,44029.0
4,2019-04-18 19:36:23.325793,2417786714126835,1,2019-04-19 06:28:35.369393,39132.0,10.870000,1,44029.0


# Primer prueba con XGBoost

In [59]:
windows2

,max_date,min_date,n,window_nr
0,2019-04-21,2019-04-18,1,1
1,2019-04-22,2019-04-19,1,2
2,2019-04-23,2019-04-20,1,3
3,2019-04-24,2019-04-21,1,4


In [60]:
win = 0
max_date = windows2.loc[win,].max_date

In [61]:
#Filtramos para la primer ventana
train_auc2 = train_auc2.groupby('date').filter(lambda x: x.date < max_date)
train_auc2.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean
0,2019-04-19 18:26:55.718385,1666627460470179,1,2019-04-19 18:29:58.659713,183.0,0.050833,1,62502.5
1,2019-04-19 18:29:58.659713,1666627460470179,1,2019-04-21 05:10:20.220516,124822.0,34.672778,1,62502.5
2,2019-04-18 10:44:10.904867,2417786714126835,3,2019-04-18 14:05:58.145714,12107.0,3.363056,1,44029.0
3,2019-04-18 14:05:58.145714,2417786714126835,1,2019-04-18 19:36:23.325793,19825.0,5.506944,1,44029.0
4,2019-04-18 19:36:23.325793,2417786714126835,1,2019-04-19 06:28:35.369393,39132.0,10.870000,1,44029.0


In [62]:
#convertidor de fecha(Timestamp) a int
def convert_date_int(date):
    d = pd.to_datetime(date)
    return np.datetime64(d).astype(int)

In [64]:
train_auc2['date_int'] = train_auc2['date'].apply(convert_date_int)
train_auc2['next_date_int'] = train_auc2['next_date'].apply(convert_date_int)

In [65]:
train_auc2.head()

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean,date_int,next_date_int
0,2019-04-19 18:26:55.718385,1666627460470179,1,2019-04-19 18:29:58.659713,183.0,0.050833,1,62502.5,1555698415718385,1555698598659713
1,2019-04-19 18:29:58.659713,1666627460470179,1,2019-04-21 05:10:20.220516,124822.0,34.672778,1,62502.5,1555698598659713,1555823420220516
2,2019-04-18 10:44:10.904867,2417786714126835,3,2019-04-18 14:05:58.145714,12107.0,3.363056,1,44029.0,1555584250904867,1555596358145714
3,2019-04-18 14:05:58.145714,2417786714126835,1,2019-04-18 19:36:23.325793,19825.0,5.506944,1,44029.0,1555596358145714,1555616183325793
4,2019-04-18 19:36:23.325793,2417786714126835,1,2019-04-19 06:28:35.369393,39132.0,10.870000,1,44029.0,1555616183325793,1555655315369393


In [94]:
columns_to_xgboost = ['ref_hash','source_id'\
                      ,'ref_type','secs_to_next_mean'\
                      ,'date_int','next_date_int','secs_to_next']

In [95]:
train_auc3 = train_auc2[columns_to_xgboost]

In [96]:
train_auc3.head()

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,secs_to_next
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,183.0
1,1666627460470179,1,1,62502.5,1555698598659713,1555823420220516,124822.0
2,2417786714126835,3,1,44029.0,1555584250904867,1555596358145714,12107.0
3,2417786714126835,1,1,44029.0,1555596358145714,1555616183325793,19825.0
4,2417786714126835,1,1,44029.0,1555616183325793,1555655315369393,39132.0


In [98]:
# sacamos el ref_hash porque no aporta nada
train_auc3 = train_auc3[['source_id','ref_type','secs_to_next_mean','secs_to_next']]
train_auc3.head()

,source_id,ref_type,secs_to_next_mean,secs_to_next
0,1,1,62502.5,183.0
1,1,1,62502.5,124822.0
2,3,1,44029.0,12107.0
3,1,1,44029.0,19825.0
4,1,1,44029.0,39132.0


In [99]:
#Separamos X y Y
X = train_auc3.loc[:,train_auc3.columns != 'secs_to_next']

In [100]:
X.head()

,source_id,ref_type,secs_to_next_mean
0,1,1,62502.5
1,1,1,62502.5
2,3,1,44029.0
3,1,1,44029.0
4,1,1,44029.0


In [101]:
y = train_auc3.loc[:,train_auc3.columns == 'secs_to_next']
y.head()

,secs_to_next
0,183.0
1,124822.0
2,12107.0
3,19825.0
4,39132.0


In [106]:
print("(reg tot,features tot), ( reg X, feat X), (reg Y, feat Y)")
train_auc3.shape,X.shape,y.shape

(reg tot,features tot), ( reg X, feat X), (reg Y, feat Y)


((61008, 4), (61008, 3), (61008, 1))

In [107]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=1)

In [108]:
X_train.shape ,X_test.shape ,y_train.shape ,y_test.shape

((42705, 3), (18303, 3), (42705, 1), (18303, 1))

### Hiper-parámetros
- learning_rate: tasa de aprendizaje
- max_depth: máxima profundidad de cada árbol(default=6)
- subsample: porcentaje de muestras usadas para cada árbol (valor muy bajo, posible underfitting)
- colsample_bytree: porcentaje de features usadas para cada árbol (valores muy alto, posible overfitting)
- n_estimators: cantidad de árboles a construir.
- objective: función de error a utilizar (algunas: reg:linear para regresión, reg:logistic o binary:logistic para clasificación)
### Parámetros de regularización:

- gamma: umbral para hacer split basado en la reducción de error de hacer el nuevo split.
- alpha: regularización para los pesos de las hojas. Un valor más alto genera una mayor regularización.
- lambda: similar alpha pero para la sintonia fina.
### Otros parámetros
- Verbosite: si muestra mensajes de info,aviso,error,debug

In [109]:
# Regressor
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.95, learning_rate = 0.42,
                max_depth = 6, alpha = 20, n_estimators = 300,verbosity=1)

In [110]:
xg_reg.fit(X_train,y_train)

[21:36:51] WARNING: src/objective/regression_obj.cu:144: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=20, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.95, gamma=0,
       importance_type='gain', learning_rate=0.42, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [111]:
preds = xg_reg.predict(X_test)

In [122]:
mse = mean_squared_error(y_test, preds)
print("mean_squared_error")
print(str(mse))
#rmse = np.sqrt(mse)
#print("RMSE: %f" % (rmse))
print("mean_squared_error NICO")
print("3400688504.3380723")
print(X_test.head(1))
xg_reg.feature_importances_
#RMSE con :
#xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 10)regresor lineal,
#                colsample_bytree=0.3, learning_rat=0.1,max_depth=1-,n_estimators=10

mean_squared_error
3483895016.745909
mean_squared_error NICO
3400688504.3380723
       source_id  ref_type  secs_to_next_mean
31030          3         1            80053.0


array([0.10515303, 0.03063782, 0.8642092 ], dtype=float32)

# Otros casos

In [117]:
X_train.describe()

,source_id,ref_type,secs_to_next_mean
count,42705.000000,42705.000000,42705.000000
mean,1.590516,1.647137,35475.281307
std,1.532023,1.861214,36349.752257
min,0.000000,1.000000,329.333333
25%,1.000000,1.000000,14609.217391
50%,1.000000,1.000000,26588.000000
75%,3.000000,1.000000,43702.250000
max,9.000000,7.000000,724842.000000


In [126]:
train_auc3.head(3)

,source_id,ref_type,secs_to_next_mean,secs_to_next
0,1,1,62502.5,183.0
1,1,1,62502.5,124822.0
2,3,1,44029.0,12107.0


## Armo un feature en base a el horario en el que se realiza la subasta
Si la subasta se realiza en el horario de 20 ~ 2 hs entonces = 1 sino = 0

In [138]:
train_auc2.head(2)

,date,ref_hash,source_id,next_date,secs_to_next,hs_to_next,ref_type,secs_to_next_mean,date_int,next_date_int
0,2019-04-19 18:26:55.718385,1666627460470179,1,2019-04-19 18:29:58.659713,183.0,0.050833,1,62502.5,1555698415718385,1555698598659713
1,2019-04-19 18:29:58.659713,1666627460470179,1,2019-04-21 05:10:20.220516,124822.0,34.672778,1,62502.5,1555698598659713,1555823420220516


In [151]:
train_auc2.head(2)
auc4 = train_auc2[columns_to_xgboost_2]
auc4.tail(2)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,date,secs_to_next,hs_to_next
61006,9205507390206506601,1,7,9910.074074,1555779334364999,1555781579555232,2019-04-20 16:55:34.364999,2245.0,0.623611
61007,9205507390206506601,0,7,9910.074074,1555781579555232,1555824264242442,2019-04-20 17:32:59.555232,42685.0,11.856944


In [171]:
def alta_demanda(date):
    #print(reg.ref_hash)
    hs = date.hour
    #print("horas"+str(hs))
    if (hs >= 20) | (hs == 1| hs == 2):
        return 1
    return 0

In [172]:
auc4['alta_demanda'] = auc4['date'].apply(alta_demanda)

In [183]:
t = auc4[['date','alta_demanda']][(auc4['alta_demanda'] == 1)]
t.sort_values(by='date').head()

,date,alta_demanda
39374,2019-04-18 20:00:01.804402,1
2624,2019-04-18 20:00:03.506865,1
19683,2019-04-18 20:00:04.282598,1
21792,2019-04-18 20:00:05.363788,1
12602,2019-04-18 20:00:09.181867,1


In [195]:
#Agregamos features
columns_to_xgboost_4 = ['ref_hash','source_id'\
                      ,'ref_type','secs_to_next_mean'\
                      ,'date_int','next_date_int','secs_to_next','alta_demanda']

In [196]:
auc4_train = auc4[columns_to_xgboost_4]
auc4_train.head(3)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,secs_to_next,alta_demanda
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,183.0,0
1,1666627460470179,1,1,62502.5,1555698598659713,1555823420220516,124822.0,0
2,2417786714126835,3,1,44029.0,1555584250904867,1555596358145714,12107.0,0


In [169]:
#2417786714126835
alta_demanda(auc4.loc[7,:]['date'])

horas5


0

In [148]:
alta_demanda(auc4.loc[1,:])

1666627460470179


0

In [188]:
auc4_train.head(2)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,date,secs_to_next,hs_to_next,alta_demanda
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,2019-04-19 18:26:55.718385,183.0,0.050833,0
1,1666627460470179,1,1,62502.5,1555698598659713,1555823420220516,2019-04-19 18:29:58.659713,124822.0,34.672778,0


In [197]:
#Separo X e Y
X4 = auc4_train.loc[:,auc4_train.columns != 'secs_to_next']
y4 = auc4_train.loc[:,auc4_train.columns == 'secs_to_next']
X_train4,X_test4,y_train4,y_test4 = train_test_split(X4,y4,test_size=0.3,random_state=1)
X_train4.shape ,X_test4.shape ,y_train4.shape ,y_test4.shape

((42705, 7), (18303, 7), (42705, 1), (18303, 1))

In [198]:
xg_reg

XGBRegressor(alpha=20, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.95, gamma=0,
       importance_type='gain', learning_rate=0.42, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [199]:
xg_reg.fit(X_train4,y_train4)
preds4 = xg_reg.predict(X_test4)
mse = mean_squared_error(y_test4, preds4)


[23:10:58] WARNING: src/objective/regression_obj.cu:144: reg:linear is now deprecated in favor of reg:squarederror.


In [200]:
mse = mean_squared_error(y_test, preds)
print("mean_squared_error")
print(str(mse))

print("mean_squared_error NICO")
print("3400688504.3380723")
print(X_test.head(1))
xg_reg.feature_importances_

mean_squared_error
3483895016.745909
mean_squared_error NICO
3400688504.3380723
       source_id  ref_type  secs_to_next_mean
31030          3         1            80053.0


array([0.00093625, 0.00074826, 0.00070856, 0.13675292, 0.16196215,
       0.6344166 , 0.06447531], dtype=float32)

In [202]:
mse = mean_squared_error(y_test4, preds4)
print("mean_squared_error")
print(str(mse))

print("mean_squared_error NICO")
print("3400688504.3380723")
print(X_test4.head(1))
xg_reg.feature_importances_

mean_squared_error
12257367.756658535
mean_squared_error NICO
3400688504.3380723
                  ref_hash  source_id  ref_type  secs_to_next_mean  \
31030  5373572805145747380          3         1            80053.0   

               date_int     next_date_int  alta_demanda  
31030  1555640719316765  1555805439843987             0  


array([0.00093625, 0.00074826, 0.00070856, 0.13675292, 0.16196215,
       0.6344166 , 0.06447531], dtype=float32)

### Armo un feature en base a el horario en el que se realiza la subasta
Como los picos de subasta se realizan entre 1 ~ 2 entonces ponemos más peso a esos casos
Ej.: [1,2] = 0.75 ; [20,0] = 0.5 ; [3,20] = 0.25


In [204]:
auc4.head(2)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,date,secs_to_next,hs_to_next,alta_demanda
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,2019-04-19 18:26:55.718385,183.0,0.050833,0
1,1666627460470179,1,1,62502.5,1555698598659713,1555823420220516,2019-04-19 18:29:58.659713,124822.0,34.672778,0


In [224]:
def pico_demanda(date):
    hs = date.hour
    if (hs == 1) | (hs == 2):
        return 0.75
    else:
        if (hs >= 3) & (hs < 20):
            return 0.25
    return 0.5


In [225]:
auc4['pico_demanda'] = auc4['date'].apply(pico_demanda)

In [232]:
#Agregamos features
columns_to_xgboost_4 = ['ref_hash','source_id'\
                      ,'ref_type','secs_to_next_mean'\
                      ,'date_int','next_date_int','secs_to_next','alta_demanda','pico_demanda']
auc4_train = auc4[columns_to_xgboost_4]

In [233]:
auc4_train.head(1)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,secs_to_next,alta_demanda,pico_demanda
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,183.0,0,0.25


In [234]:
#Separo X e Y
X4 = auc4_train.loc[:,auc4_train.columns != 'secs_to_next']
y4 = auc4_train.loc[:,auc4_train.columns == 'secs_to_next']
X_train4,X_test4,y_train4,y_test4 = train_test_split(X4,y4,test_size=0.3,random_state=1)
X_train4.shape ,X_test4.shape ,y_train4.shape ,y_test4.shape

((42705, 8), (18303, 8), (42705, 1), (18303, 1))

In [236]:
xg_reg.fit(X_train4,y_train4)
preds4 = xg_reg.predict(X_test4)
mse = mean_squared_error(y_test4, preds4)

[23:29:38] WARNING: src/objective/regression_obj.cu:144: reg:linear is now deprecated in favor of reg:squarederror.


In [237]:
mse = mean_squared_error(y_test4, preds4)
print("mean_squared_error")
print(str(mse))

print("mean_squared_error NICO")
print("3400688504.3380723")
print(X_test4.head(1))
xg_reg.feature_importances_

mean_squared_error
10338789.460596887
mean_squared_error NICO
3400688504.3380723
                  ref_hash  source_id  ref_type  secs_to_next_mean  \
31030  5373572805145747380          3         1            80053.0   

               date_int     next_date_int  alta_demanda  pico_demanda  
31030  1555640719316765  1555805439843987             0          0.75  


array([7.4479845e-04, 5.0882361e-04, 5.5570691e-04, 1.3877456e-01,
       1.9706117e-01, 6.5669340e-01, 4.2920336e-03, 1.3693740e-03],
      dtype=float32)

### Armo un feature en base a el dia de la semana
Los picos de eventos se producen los días martes y miércoles
Entonces si el día de aparición se produce un día martes o miércoles => 1 sino 0


In [257]:
def ponderacion_dia(date):
    #print(date)
    day = date.day_name()
    if (day == 'Tuesday') | (day == 'Wednesday'):
        return 1
    return 0

In [264]:
#for i in range(1,3):
 #   print(i)
    #ponderacion_dia(auc4.loc[i,:]['date'])

In [261]:
auc4['pond_dia'] = auc4['date'].apply(ponderacion_dia)
auc4.head(3)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,date,secs_to_next,hs_to_next,alta_demanda,pico_demanda,pond_dia
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,2019-04-19 18:26:55.718385,183.0,0.050833,0,0.25,0
1,1666627460470179,1,1,62502.5,1555698598659713,1555823420220516,2019-04-19 18:29:58.659713,124822.0,34.672778,0,0.25,0
2,2417786714126835,3,1,44029.0,1555584250904867,1555596358145714,2019-04-18 10:44:10.904867,12107.0,3.363056,0,0.25,0


### Afino el feature en base a el horario en el que se realiza la subasta
Separa en rangos de horarios ponderando cuando se produjo la subasta
Ej.: [1,2] = 0.90 ; [20,0] u 3 = 0.70 ; [11,20] u 4 = 0.5 ;[5,10] = 0.20

In [268]:
def pond_hora_ext(date):
    hs = date.hour
    r0 = range(20,24)
    r1 = range(11,20)
    r2 = range(5,11)
    if(hs in r2):
        return 0.2
    else:
        if(hs == 0) | (hs == 3)| (hs in r0):
            return 0.7
        else:
            if(hs == 4) | (hs in r1):
                return 0.5
    return 0.9

In [269]:
auc4['pond_hora_ext'] = auc4['date'].apply(pond_hora_ext)

In [270]:
auc4[['date','pond_hora_ext']]

,date,pond_hora_ext
0,2019-04-19 18:26:55.718385,0.5
1,2019-04-19 18:29:58.659713,0.5
2,2019-04-18 10:44:10.904867,0.2
3,2019-04-18 14:05:58.145714,0.5
4,2019-04-18 19:36:23.325793,0.5
5,2019-04-19 06:28:35.369393,0.2
6,2019-04-19 23:10:05.199846,0.7
7,2019-04-20 05:12:44.952333,0.2
8,2019-04-20 22:30:00.803623,0.7
9,2019-04-18 00:32:23.647778,0.7


In [271]:
auc4.head(1)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,date,secs_to_next,hs_to_next,alta_demanda,pico_demanda,pond_dia,pond_hora_ext
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,2019-04-19 18:26:55.718385,183.0,0.050833,0,0.25,0,0.5


In [282]:
#Agregamos features
columns_to_xgboost_4 = ['ref_hash','source_id'\
                      ,'ref_type','secs_to_next_mean'\
                      ,'date_int','next_date_int','secs_to_next','pico_demanda','pond_hora_ext']
auc4_train = auc4[columns_to_xgboost_4]

In [283]:
auc4_train.head(1)

,ref_hash,source_id,ref_type,secs_to_next_mean,date_int,next_date_int,secs_to_next,pico_demanda,pond_hora_ext
0,1666627460470179,1,1,62502.5,1555698415718385,1555698598659713,183.0,0.25,0.5


In [279]:
 #Separo X e Y
X5 = auc4_train.loc[:,auc4_train.columns != 'secs_to_next']
y5 = auc4_train.loc[:,auc4_train.columns == 'secs_to_next']
X_train5,X_test5,y_train5,y_test5 = train_test_split(X5,y5,test_size=0.3,random_state=1)
X_train5.shape ,X_test5.shape ,y_train5.shape ,y_test5.shape

((42705, 8), (18303, 8), (42705, 1), (18303, 1))

In [280]:
xg_reg.fit(X_train5,y_train5)
preds5 = xg_reg.predict(X_test5)
mse = mean_squared_error(y_test5, preds5)

[00:14:25] WARNING: src/objective/regression_obj.cu:144: reg:linear is now deprecated in favor of reg:squarederror.


In [281]:
mse = mean_squared_error(y_test5, preds5)
print("mean_squared_error")
print(str(mse))

print("mean_squared_error NICO")
print("3400688504.3380723")
print(X_test5.head(1))
xg_reg.feature_importances_

mean_squared_error
9580514.500022912
mean_squared_error NICO
3400688504.3380723
                  ref_hash  source_id  ref_type  secs_to_next_mean  \
31030  5373572805145747380          3         1            80053.0   

               date_int     next_date_int  pico_demanda  pond_hora_ext  
31030  1555640719316765  1555805439843987          0.75            0.9  


array([7.5699331e-04, 5.6191674e-04, 5.2870699e-04, 1.3824295e-01,
       2.0681110e-01, 6.5145433e-01, 7.6757319e-04, 8.7644759e-04],
      dtype=float32)